In [4]:
import os
import pathlib

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [5]:
data_dir = "./data/SAVEE"

filenames = tf.io.gfile.glob(str(data_dir) + '/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print('Number of total examples:', num_samples)
print('Example file tensor:', filenames[0])

Number of total examples: 480
Example file tensor: tf.Tensor(b'.\\data\\SAVEE\\JE_a08.wav', shape=(), dtype=string)


In [6]:
test_file = tf.io.read_file(f"{data_dir}/DC_a01.wav")
test_audio, _ = tf.audio.decode_wav(contents=test_file)
test_audio.shape

TensorShape([160868, 1])

In [7]:
def decode_audio(audio_binary):
  # Decode WAV-encoded audio files to `float32` tensors, normalized
  # to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
  audio, _ = tf.audio.decode_wav(contents=audio_binary)
  # Since all the data is single channel (mono), drop the `channels`
  # axis from the array.
  return tf.squeeze(audio, axis=-1)

In [9]:
def get_label(file_path):
    parts = tf.strings.split( file_path, sep='_')
    print(parts)
    return parts

get_label(filenames[0])


tf.Tensor([b'.\\data\\SAVEE\\JE' b'a08.wav'], shape=(2,), dtype=string)


<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'.\\data\\SAVEE\\JE', b'a08.wav'], dtype=object)>